# how to use the task1 dataset tfrecords 

Probably it will be necessary to train your model in 'Gogle Colaboratory' (colab) using the available GPU.
So we start with some instructions to can use your task1 dataset in colab.

Use tensorflow 2

In [ ]:
import os
import tensorflow as tf
import numpy as np
from tensorflow import keras
#import IPython.display as display

print(tf.__version__)


## Create tfrecords dataset

**tf.data.Dataset** is tensorflow data input pipeline to used while training neural network models.
**Tfrecords** are well integrated in the tf.data.Dataset framework.

In [ ]:
tfrecordsDir = '../input/gray2Recs/'
trainNumbers = list(range(34))
validNumbers = [34,36,37,38,39]
trainfiles = [tfrecordsDir + 'images_gray_'+str(i) + '.tfrecords' for i in trainNumbers]
validfiles = [tfrecordsDir + 'images_gray_'+str(i) + '.tfrecords' for i in validNumbers]

train_raw_image_dataset = tf.data.TFRecordDataset(trainfiles)
valid_raw_image_dataset = tf.data.TFRecordDataset(validfiles)

How many examples are in the tfrecord

In [ ]:
count =0
for i in train_raw_image_dataset:
  count +=1
print(count)

## Decode tfrecords

In [ ]:
# Create a dictionary describing the features.
image_feature_description = {
            'chinaprov': tf.io.FixedLenFeature([], tf.int64),
            'letter': tf.io.FixedLenFeature([], tf.int64),
            'seq0':  tf.io.FixedLenFeature([], tf.int64),
            'seq1': tf.io.FixedLenFeature([], tf.int64),
            'seq2':  tf.io.FixedLenFeature([], tf.int64),
            'seq3': tf.io.FixedLenFeature([], tf.int64),
            'seq4':  tf.io.FixedLenFeature([], tf.int64),
            'image_raw': tf.io.FixedLenFeature([], tf.string),
        }


In [ ]:
def _parse_image_function(example_proto):
  # Parse the input tf.Example proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, image_feature_description)


In [ ]:
train_parsed_image_dataset = train_raw_image_dataset.map(_parse_image_function)
val_parsed_image_dataset = valid_raw_image_dataset.map(_parse_image_function)

### **look** at one example

In [ ]:
import IPython.display as display
it = iter(train_parsed_image_dataset)
data_parsed =next(it)
image = data_parsed['image_raw'].numpy()
display.display(display.Image(data=image))
img = tf.image.decode_png(data_parsed['image_raw'])
print(img.numpy().shape)
print(img.numpy().dtype)

 ## Create inputs and labels from raw images

In [ ]:
num_chinaprov = 34
num_letter = 34
num_seq = 34



def buildLabels(parsed_data):
  chinaprov = parsed_data['chinaprov']
  chinaprov_enc = tf.keras.backend.one_hot(chinaprov,num_chinaprov)
  letter = parsed_data['letter']
  letter_enc = tf.keras.backend.one_hot(letter, num_letter)
  seq0 = parsed_data['seq0']
  seq0_enc = tf.keras.backend.one_hot(seq0, num_seq)
  seq1 = parsed_data['seq1']
  seq1_enc = tf.keras.backend.one_hot(seq1, num_seq)
  seq2 = parsed_data['seq2']
  seq2_enc = tf.keras.backend.one_hot(seq2, num_seq)
  seq3 = parsed_data['seq3']
  seq3_enc = tf.keras.backend.one_hot(seq3, num_seq)
  seq4  = parsed_data['seq4']
  seq4_enc = tf.keras.backend.one_hot(seq4, num_seq)
  label = tf.stack([chinaprov_enc, letter_enc, seq0_enc, seq1_enc, seq2_enc, seq3_enc, seq4_enc])
  return label



def buildInputsLabels(parsed_data):
  img = tf.image.decode_png(parsed_data['image_raw'])
  img = tf.dtypes.cast(img, dtype = tf.float32)
  labels = buildLabels(parsed_data)   
  return   img,  labels

In [ ]:
trainDataset = train_parsed_image_dataset.map(buildInputsLabels).shuffle(2000)
valDataset = val_parsed_image_dataset.map(buildInputsLabels).shuffle(2000)

### get one example, inputs, label from the dataset

In [ ]:
newIt = iter(trainDataset)
data =next(newIt)
data[1].shape


In [ ]:
import pandas as pd
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, BatchNormalization, Conv2D, Dense
from tensorflow.keras.layers import MaxPooling2D, Activation, Flatten, Dropout
from tensorflow.keras.layers import LeakyReLU, Reshape, Concatenate
from tensorflow.keras import regularizers

In [ ]:
batchsize=33
batchDataset = trainDataset.batch(batchsize)

valbatch=33
val_batch = valDataset.batch(valbatch)

In [ ]:
output_nodes = data[1].shape[1]
n_classes = data[1].shape[0]

In [ ]:
data[0].shape

In [ ]:
inputs = Input(shape=data[0].shape,name='inputs')

layer=Conv2D(10, kernel_size=(9,9),data_format='channels_last',strides=2)(inputs)
layer=LeakyReLU(alpha=0.1)(layer)
layer=BatchNormalization()(layer)
layer=MaxPooling2D(pool_size=(2, 2),padding='same')(layer)
layer=Dropout(0.4)(layer)

layer=Conv2D(16, kernel_size=(9,9))(layer)
layer=LeakyReLU(alpha=0.1)(layer)
layer=BatchNormalization()(layer)
layer=MaxPooling2D(pool_size=(2, 2),padding='same')(layer)
layer1=Dropout(0.4)(layer)

Clayer1=Flatten()(layer1)

layer=Conv2D(32, kernel_size=(9, 9), padding='same')(layer1)
layer=LeakyReLU(alpha=0.1)(layer)
layer=BatchNormalization()(layer)
layer=MaxPooling2D(pool_size=(2, 2),padding='same')(layer)
layer=Dropout(0.4)(layer)

layer=Conv2D(64, kernel_size=(9, 9), padding='same')(layer)
layer=LeakyReLU(alpha=0.1)(layer)
layer=BatchNormalization()(layer)
layer=MaxPooling2D(pool_size=(2, 2),padding='same')(layer)
layer2=Dropout(0.4)(layer)

Clayer2=Flatten()(layer2)

layer=Conv2D(128, kernel_size=(9, 9), padding='same')(layer2)
layer=LeakyReLU(alpha=0.1)(layer)
layer=BatchNormalization()(layer)
layer=MaxPooling2D(pool_size=(2, 2),padding='same')(layer)
layer=Dropout(0.4)(layer)

layer=Conv2D(160, kernel_size=(9, 9), padding='same')(layer)
layer=LeakyReLU(alpha=0.1)(layer)
layer=BatchNormalization()(layer)
layer=MaxPooling2D(pool_size=(2, 2),padding='same')(layer)
layer3=Dropout(0.4)(layer)

Clayer3=Flatten()(layer3)

layer=Conv2D(190, kernel_size=(9, 9), padding='same')(layer3)
layer=LeakyReLU(alpha=0.1)(layer)
layer=BatchNormalization()(layer)
layer=MaxPooling2D(pool_size=(2, 2),padding='same')(layer)
layer=Dropout(0.4)(layer)

layer=Conv2D(220, kernel_size=(9, 9), padding='same')(layer)
layer=LeakyReLU(alpha=0.1)(layer)
layer=BatchNormalization()(layer)
layer=MaxPooling2D(pool_size=(2, 2),padding='same')(layer)
layer4=Dropout(0.4)(layer)

Clayer4=Flatten()(layer4)


merge=Concatenate()([Clayer1, Clayer2, Clayer3, Clayer4])

predlayer=Flatten()(merge)
predlayer=Dense(320)(predlayer)
predlayer=LeakyReLU(alpha=0.1)(predlayer)
predlayer=BatchNormalization()(predlayer)
predlayer=Dropout(0.5)(predlayer)

predlayer=Dense(280)(predlayer)
predlayer=LeakyReLU(alpha=0.1)(predlayer)
predlayer=BatchNormalization()(predlayer)
predlayer=Dropout(0.5)(predlayer)

predlayer=Dense(250)(predlayer)
predlayer=LeakyReLU(alpha=0.1)(predlayer)
predlayer=BatchNormalization()(predlayer)
predlayer=Dropout(0.5)(predlayer)

output_layer    = Dense(7*34)(predlayer)
output_reshape  = Reshape((7,34))(output_layer)

In [ ]:
model = Model(inputs = inputs, outputs = output_reshape)

In [ ]:
model.summary()

In [ ]:
def train(NUM_EPOCHS=30):
  model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), optimizer="Adamax", 
                  metrics=["accuracy"])

  H = model.fit(batchDataset,validation_data=(val_batch), epochs=NUM_EPOCHS)
  return H

def predict(x):
  return np.argmax(model.predict(x),axis=-1)


def predict_prob(x):
  return model.predict(x)

def evaluate(x,y,batches=batchsize):
  return model.evaluate(x,y)

In [ ]:
history=train(35)

In [ ]:
dic=history.history

In [ ]:
# Save the model
model.save('deep_model.h5')


In [ ]:
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
from matplotlib.pyplot import figure


In [ ]:
figure(num=None, figsize=(12, 7), dpi=80, facecolor='w', edgecolor='k')
plt.plot(dic["accuracy"]);
plt.plot(dic["val_accuracy"]);
plt.savefig("/acc.png")

In [ ]:
figure(num=None, figsize=(12, 7), dpi=80, facecolor='w', edgecolor='k')
plt.plot(dic["loss"]);
plt.plot(dic["val_loss"]);
plt.savefig("/loss.png")

In [ ]:
newIt = iter(val_batch)
data =next(newIt)
data[0]

In [ ]:
predict(data[0])

In [ ]:
np.argmax(data[1].numpy(),axis=-1)

In [ ]:
evaluate(data[0],data[1])